# 该程序是用来将GID数据集的vis_png转换成masks__png的

In [1]:
import numpy as np
from PIL import Image
import glob
import time
import torch
from matplotlib import pyplot as plt 

In [2]:
#转换数据集
"""因为，可以看数据集发现，masks_png都是0-1的小数，没有办法训练，所以这里需要转换一下数据集"""
"""给的代码示例是一次转换一张，现在试试一次性全部转换"""
#类别数 背景-0、建筑-1、农田-2、森林-3、草地-4、水-5。
GID_COLOR_MAP = dict(
    Background=(0, 0, 0), #0-背景-黑色
    Building=(255, 0, 0), #1-建筑-红色
    Farmland=(0, 255, 0), #2-农田-绿色
    Forest=(0, 0, 255), #3-森林-蓝色
    Meadow=(255, 255, 0),#4-草地-黄色
    Water=(0, 0, 255),#5-水-蓝色
)
    
GID_COLORMAP = list(GID_COLOR_MAP.values())
GID_CLASSES = list(GID_COLOR_MAP.keys())

In [3]:
#############用列表来存一个RGB和一个类别的对应################
def GID_colormap2label(GID_COLORMAP):   
    colormap2label = torch.zeros(256**3,dtype = torch.long)
    for i, colormap in enumerate(GID_COLORMAP):
        colormap2label[(colormap[0]*256+colormap[1])*256+colormap[2]] = i
    return colormap2label

#############给定那个列表，和vis_png然后生成masks_png################
def GID_label_indices(vis_png, colormap2label):
    colormap = vis_png.astype('int32')
    idx = (colormap[:,:,0]*256+colormap[:,:,1])*256+colormap[:,:,2]
    return colormap2label[idx]

In [4]:
colormap2label = GID_colormap2label(GID_COLORMAP)  #生成标签的那个查找表列表

In [5]:
vis_path = '/root/master/ATL/dataset/GID/vis_png/GF2_PMS1__L1A0000564539-MSS1_label.tif' #vis_png的图片路径
vis_img = np.array(Image.open(vis_path)) #打开vis_png
masks_path = '/root/master/ATL/dataset/GID/masks_png/'+vis_path[37:]
masks_png = GID_label_indices(vis_img,colormap2label).numpy()
color_img = Image.fromarray(np.uint8(masks_png)) #从np的array格式转换成PIL格式
color_img.save(masks_path)

In [6]:
#把masks_png转换成vis_png
def render(mask_path, vis_path):
    new_mask = np.array(Image.open(mask_path)).astype(np.uint8) #把原来的图片读出来，存成uint8的格式
    cm = np.array(GID_COLORMAP).astype(np.uint8) #取出COLOR_MAP中的值，存到列表
    color_img = cm[new_mask]  #这里就变成1024*1024*3的了？ why？？
    color_img = Image.fromarray(np.uint8(color_img)) #从np的array格式转换成PIL格式
    color_img.save(vis_path)

#把vis_png转换成masks_png
def create_masks_png(vis_path, masks_path):
    vis_img = np.array(Image.open(vis_path)) #打开vis_png
    masks_png = GID_label_indices(vis_img,colormap2label).numpy()
    color_img = Image.fromarray(np.uint8(masks_png)) #从np的array格式转换成PIL格式
    color_img.save(masks_path)

In [7]:
#数据集的地址    
vis_file_path = '/root/master/ATL/dataset/GID/vis_png/'
#数据集的地址列表
vis_list = np.array(sorted(glob.glob(vis_file_path + '*.tif'))) #把这个文件夹中的图片的地址，以有序的列表形式存放    
masks_path = '/root/master/ATL/dataset/GID/masks_png/'

In [8]:
# create_masks_png(vis_list[0], masks_path+vis_list[0][37:])

# render(masks_path+vis_list[0][37:],masks_path+'1'+vis_list[0][37:])

In [10]:
if __name__ == '__main__':
    print('vis_png 转换为 masks_png ing......')
    start = time.time()
    for i in range(len(vis_list)):
        create_masks_png(vis_list[i], masks_path+vis_list[i][37:])
    end = time.time()
    print(f'共转换vis_png {len(vis_list)} 张,耗时 {(end-start)} 秒')

vis_png 转换为 masks_png ing......
共转换vis_png 15 张,耗时 24.926617860794067 秒
